In [0]:
 
dbutils.fs.mount(source = "wasbs://2023@shellidastorage.blob.core.windows.net/",
                 mount_point ="/mnt/input/",
                 extra_configs = {"fs.azure.account.key.shellidastorage.blob.core.windows.net":dbutils.secrets.get(scope = "testscope",key="adlssecrets")})

True

In [0]:
dbutils.secrets.listScopes()

[SecretScope(name='testscope')]

In [0]:
 %fs ls /mnt/input/retail

path,name,size,modificationTime
dbfs:/mnt/input/retail/nation.csv,nation.csv,2240,1695721047000
dbfs:/mnt/input/retail/region.csv,region.csv,413,1695721134000


In [0]:
from pyspark.sql.types import *
schema = StructType([StructField("lsoa_code", StringType(), True),\
                         StructField("borough", StringType(), True),\
                         StructField("major_category", StringType(), True),\
                         StructField("minor_category", StringType(), True),\
                         StructField("value", StringType(), True),\
                         StructField("year", StringType(), True),\
                         StructField("month", StringType(), True)])

In [0]:

trimmedDF = fileStreamDF.select(
                                      fileStreamDF.borough,
                                      fileStreamDF.year,
                                      fileStreamDF.month,
                                      fileStreamDF.value
                                      )\
                             .withColumnRenamed(
                                      "value",
                                      "convictions"
                                      )


query = trimmedDF.writeStream\
                      .outputMode("append")\
                      .format("console")\
                      .option("truncate", "false")\
                      .option("numRows", 30)\
                      .start()

query = trimmedDF.writeStream\
                      .outputMode("append")\
                      .format("parquet")\
                      .option("truncate", "false")\
                      .option("checkpointLocation", "/mnt/input/checkpoint") \
                      .option("path", "/mnt/input/target/crimeout") \
                      .start()